<a href="https://colab.research.google.com/github/Nikhileswar-Komati/100D_ML/blob/master/Copy_of_Go_Vs_Python_FILE_IO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory

profile = ExecutionProfile(
    load_balancing_policy=WhiteListRoundRobinPolicy(['127.0.0.1']),
    retry_policy=DowngradingConsistencyRetryPolicy(),
    consistency_level=ConsistencyLevel.LOCAL_QUORUM,
    serial_consistency_level=ConsistencyLevel.LOCAL_SERIAL,
    request_timeout=15,
    row_factory=tuple_factory
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()

print(session.execute("SELECT release_version FROM system.local").one())

In [ ]:
import csv
import os
import time
import pandas as pd

# df = pd.read_csv("from_2012.csv")
# print(df.shape)
csv_reader1 = csv.reader(open("from_2012.csv"))
csv_reader2 = csv.reader(open("from_2012.csv"))
csv_reader3 = csv.reader(open("from_2012.csv"))
a = time.time()

with open('file1.csv', 'w') as csvfile1:
    csv_writer1 = csv.writer(csvfile1)
    for row in csv_reader1:
        csv_writer1.writerow(row)

b = time.time()

# with open('file2.csv', 'w') as csvfile2:
#     csv_writer2 = csv.writer(csvfile2)

#     i = 0
#     rows = []
#     for row in csv_reader2:
#         i +=1
#         rows.append(row)
#         if i==10000:
#             i = 0
#             csv_writer2.writerows(rows)
#             rows = []

# c = time.time()

# chunks = c-b
# single = b-a
# # print(c-b)
# print(chunks, single, chunks>single)

# a = time.time()

# with open('file1.csv', 'w') as csvfile1:
#     csv_writer1 = csv.writer(csvfile1)
#     csv_writer1.writerows(csv_reader3)

# b = time.time()

print(b-a)

In [ ]:
import csv
import os
import time



steps = [2, 10, 20]

ind = 0
for step in steps:
    csv_reader = csv.reader(open("from_2012.csv"))
    b = time.time()
    file_name = 'file{}.csv'.format(ind)
    with open(file_name, 'w') as csvfile2:
        csv_writer2 = csv.writer(csvfile2)

        i = 0
        rows = []
        for row in csv_reader:
            i +=1
            rows.append(row)
            if i==step:
                i = 0
                csv_writer2.writerows(rows)
                rows = []
    ind += 1
    c = time.time()

    chunks = c-b
    print(chunks, file_name)


In [ ]:
package main

import (
	"encoding/csv"
	"fmt"
	"log"
	"os"
	"time"
)

func readCsvFile(filePath string) [][]string {
	f, err := os.Open(filePath)
	if err != nil {
		log.Fatal("Unable to read input file "+filePath, err)
	}
	defer f.Close()

	csvReader := csv.NewReader(f)
	records, err := csvReader.ReadAll()
	if err != nil {
		log.Fatal("Unable to parse file as CSV for "+filePath, err)
	}

	return records
}

func writeCsvFile(records [][]string, filePath string) {
	f, err := os.Create(filePath)
	if err != nil {
		log.Fatalf("failed creating file: %s", err)
	}
	defer f.Close()

	csvwriter := csv.NewWriter(f)
	for _, record := range records {
		_ = csvwriter.Write(record)
	}
	csvwriter.Flush()
}

func writeAllCsvFile(records [][]string, filePath string) {
	f, err := os.Create(filePath)
	if err != nil {
		log.Fatalf("failed creating file: %s", err)
	}
	defer f.Close()

	csvwriter := csv.NewWriter(f)
	csvwriter.WriteAll(records)
}

func main() {
	records := readCsvFile("from_2012.csv")
	start := time.Now()
	writeCsvFile(records, "gofile.csv")
	end := time.Now()
	fmt.Println(end.Sub(start))
	start1 := time.Now()
	writeAllCsvFile(records, "goallfile.csv")
	end1 := time.Now()
	fmt.Println(end1.Sub(start1))
}


In [ ]:
package main

import (
	"fmt"
	"github.com/go-gota/gota/dataframe"
	"gonum.org/v1/gonum/mat"
	// "reflect"
	"github.com/pa-m/sklearn/preprocessing"
	"strings"
	"io/ioutil"
)

type matrix struct {
	dataframe.DataFrame
}

func (m matrix) At(i, j int) float64 {
	return m.Elem(i, j).Float()
}

func (m matrix) T() mat.Matrix {
	return mat.Transpose{Matrix: m}
}

func getNumericalColumns(df dataframe.DataFrame) dataframe.DataFrame{
	list_of_datatypes := df.Types()
	var numerical_datatype_indexes []int
	for indx, datatype := range list_of_datatypes {
		if datatype == "float" || datatype == "int" {
			numerical_datatype_indexes = append(numerical_datatype_indexes, indx)
		}
	}
	return df.Select(numerical_datatype_indexes)
}

func data_normalization_function(data dataframe.DataFrame, norm_type string, composite_columns []string) {
	fmt.Println("In Data Normalization") 
	norm_type = strings.ToLower(norm_type)
	data_without_time := data.Drop(composite_columns)
	data_num := getNumericalColumns(data_without_time)
	data_num_matrix := matrix{data_num}
	var scaled_num_data *mat.Dense
	if norm_type == "minmax" {
		scaler := preprocessing.NewMinMaxScaler([]float64{0, 1})
		scaled_num_data, _ = scaler.FitTransform(data_num_matrix, nil)
	} else if norm_type == "standard" {
		scaler := preprocessing.NewStandardScaler()
		scaled_num_data, _ = scaler.FitTransform(data_num_matrix, nil)
	} else if norm_type == "maxabs" {
		scaler := preprocessing.NewMaxAbsScaler()
		scaled_num_data, _ = scaler.FitTransform(data_num_matrix, nil)
	} else if norm_type == "robust" {
		scaler := preprocessing.NewRobustScaler(true, false, nil)
		scaled_num_data, _ = scaler.FitTransform(data_num_matrix, nil)
	}
	fmt.Println(scaled_num_data)
	scaled_num_data_frame := dataframe.LoadMatrix(scaled_num_data)
	num_column_names := data_num.Names()
	scaled_num_data_frame.SetNames(num_column_names...)
	fmt.Println(scaled_num_data_frame)
	df_without_num_columns := data.Drop(num_column_names)
	data = scaled_num_data_frame.CBind(df_without_num_columns)
	fmt.Println(data)
}

func main() {
	content, _ := ioutil.ReadFile("sample_go.csv")
	ioContent := strings.NewReader(string(content))

	df := dataframe.ReadCSV(ioContent)
	norm_type := "minmax"
	var composite_columns []string

	data_normalization_function(df, norm_type, composite_columns)
}